In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1763675,2021-10-19,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1763676,2021-10-20,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1763677,2021-10-21,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1763678,2021-10-22,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
45939,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
45941,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
45943,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
45945,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
45947,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
1073997,2021-10-19,Arkansas,Arkansas,5001,3450,59.00,5000,Arkansas,AR,Arkansas,State,3017804
1073999,2021-10-20,Arkansas,Arkansas,5001,3451,59.00,5000,Arkansas,AR,Arkansas,State,3017804
1074001,2021-10-21,Arkansas,Arkansas,5001,3456,59.00,5000,Arkansas,AR,Arkansas,State,3017804
1074003,2021-10-22,Arkansas,Arkansas,5001,3467,59.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1763675,2021-10-19,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1763676,2021-10-20,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1763677,2021-10-21,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1763678,2021-10-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-10-23') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
641,2021-10-23,Snohomish,Washington,63895,744.00,53061,WA,County,822083,7772.33,90.50
1280,2021-10-23,Cook,Illinois,636261,11769.00,17031,IL,County,5150233,12354.02,228.51
1918,2021-10-23,Orange,California,324288,5562.00,6059,CA,County,3175692,10211.57,175.14
2555,2021-10-23,Maricopa,Arizona,728245,12041.00,4013,AZ,County,4485414,16235.85,268.45
3192,2021-10-23,Los Angeles,California,1484896,26540.00,6037,CA,County,10039107,14791.12,264.37
...,...,...,...,...,...,...,...,...,...,...,...
1762299,2021-10-23,Wheeler,Oregon,112,1.00,41069,OR,County,1332,8408.41,75.08
1762675,2021-10-23,King,Texas,16,0.00,48269,TX,County,272,5882.35,0.00
1763020,2021-10-23,Esmeralda,Nevada,58,2.00,32009,NV,County,873,6643.76,229.10
1763361,2021-10-23,Loving,Texas,8,0.00,48301,TX,County,169,4733.73,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
641,2021-10-23,Snohomish,Washington,63895,744.00,53061,WA,County,822083,7772.33,90.50,90.50,7772.33
1280,2021-10-23,Cook,Illinois,636261,11769.00,17031,IL,County,5150233,12354.02,228.51,228.51,12354.02
1918,2021-10-23,Orange,California,324288,5562.00,6059,CA,County,3175692,10211.57,175.14,175.14,10211.57
2555,2021-10-23,Maricopa,Arizona,728245,12041.00,4013,AZ,County,4485414,16235.85,268.45,268.45,16235.85
3192,2021-10-23,Los Angeles,California,1484896,26540.00,6037,CA,County,10039107,14791.12,264.37,264.37,14791.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1762299,2021-10-23,Wheeler,Oregon,112,1.00,41069,OR,County,1332,8408.41,75.08,75.08,8408.41
1762675,2021-10-23,King,Texas,16,0.00,48269,TX,County,272,5882.35,0.00,0.00,5882.35
1763020,2021-10-23,Esmeralda,Nevada,58,2.00,32009,NV,County,873,6643.76,229.10,229.10,6643.76
1763361,2021-10-23,Loving,Texas,8,0.00,48301,TX,County,169,4733.73,0.00,0.00,4733.73


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
1123366,2021-10-23,Hale,Alabama,3113,88.00,1065,AL,County,14651,21247.70,600.64,600.64,21247.70,1
1223561,2021-10-23,Clarke,Alabama,4755,84.00,1025,AL,County,23622,20129.54,355.60,355.60,20129.54,2
317252,2021-10-23,Calhoun,Alabama,22225,488.00,1015,AL,County,113605,19563.40,429.56,429.56,19563.40,3
777593,2021-10-23,Etowah,Alabama,19825,500.00,1055,AL,County,102268,19385.34,488.91,488.91,19385.34,4
442900,2021-10-23,Jackson,Alabama,9871,184.00,1071,AL,County,51626,19120.21,356.41,356.41,19120.21,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1674939,2021-10-23,Weston,Wyoming,1027,9.00,56045,WY,County,6927,14826.04,129.93,129.93,14826.04,19
945090,2021-10-23,Johnson,Wyoming,1166,16.00,56019,WY,County,8445,13806.99,189.46,189.46,13806.99,20
1072283,2021-10-23,Sublette,Wyoming,1336,15.00,56035,WY,County,9831,13589.67,152.58,152.58,13589.67,21
1334713,2021-10-23,Lincoln,Wyoming,2682,18.00,56023,WY,County,19830,13524.96,90.77,90.77,13524.96,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
877581,2021-10-23,Lowndes,Alabama,1751,64.00,1085,AL,County,9726,18003.29,658.03,658.03,18003.29,17,1
1123366,2021-10-23,Hale,Alabama,3113,88.00,1065,AL,County,14651,21247.70,600.64,600.64,21247.70,1,2
371654,2021-10-23,Walker,Alabama,11202,370.00,1127,AL,County,63521,17635.11,582.48,582.48,17635.11,24,3
876427,2021-10-23,Crenshaw,Alabama,2452,75.00,1041,AL,County,13772,17804.24,544.58,544.58,17804.24,22,4
946818,2021-10-23,Greene,Alabama,1267,44.00,1063,AL,County,8111,15620.76,542.47,542.47,15620.76,47,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1674939,2021-10-23,Weston,Wyoming,1027,9.00,56045,WY,County,6927,14826.04,129.93,129.93,14826.04,19,19
1301142,2021-10-23,Uinta,Wyoming,3781,26.00,56041,WY,County,20226,18693.76,128.55,128.55,18693.76,5,20
1334713,2021-10-23,Lincoln,Wyoming,2682,18.00,56023,WY,County,19830,13524.96,90.77,90.77,13524.96,22,21
874119,2021-10-23,Albany,Wyoming,6217,32.00,56001,WY,County,38880,15990.23,82.30,82.30,15990.23,15,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,90.50,7772.33,27,29
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,90.50,7772.33,27,29
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,90.50,7772.33,27,29
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,90.50,7772.33,27,29
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,90.50,7772.33,27,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1759587,2021-10-19,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1759588,2021-10-20,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1759589,2021-10-21,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1759590,2021-10-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa[ '14day_case_avg' ] = usa.case_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg
1118706,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,600.64,21247.70,2,1,1.00,nan
1118707,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,600.64,21247.70,2,1,0.00,nan
1118708,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,600.64,21247.70,2,1,0.00,nan
1118709,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,600.64,21247.70,2,1,0.00,nan
1118710,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,600.64,21247.70,2,1,2.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1357317,2021-10-19,Crook,Wyoming,853,18.00,56011,WY,County,7584,11247.36,237.34,237.34,11392.41,8,23,7.00,3.86
1357318,2021-10-20,Crook,Wyoming,860,18.00,56011,WY,County,7584,11339.66,237.34,237.34,11392.41,8,23,7.00,4.07
1357319,2021-10-21,Crook,Wyoming,862,18.00,56011,WY,County,7584,11366.03,237.34,237.34,11392.41,8,23,2.00,4.07
1357320,2021-10-22,Crook,Wyoming,864,18.00,56011,WY,County,7584,11392.41,237.34,237.34,11392.41,8,23,2.00,3.43


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa[ '14day_death_avg' ] = usa.death_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
874067,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,658.03,18003.29,1,17,1.00,5.00,0.00,nan
874068,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,658.03,18003.29,1,17,0.00,4.86,0.00,nan
874069,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,658.03,18003.29,1,17,0.00,4.86,0.00,nan
874070,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,658.03,18003.29,1,17,0.00,4.14,0.00,nan
874071,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,658.03,18003.29,1,17,0.00,3.43,0.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367538,2021-10-19,Teton,Wyoming,5083,14.00,56039,WY,County,23464,21662.97,59.67,59.67,21816.40,23,2,8.00,9.07,1.00,0.07
367539,2021-10-20,Teton,Wyoming,5098,14.00,56039,WY,County,23464,21726.90,59.67,59.67,21816.40,23,2,15.00,9.86,0.00,0.07
367540,2021-10-21,Teton,Wyoming,5110,14.00,56039,WY,County,23464,21778.04,59.67,59.67,21816.40,23,2,12.00,9.79,0.00,0.07
367541,2021-10-22,Teton,Wyoming,5119,14.00,56039,WY,County,23464,21816.40,59.67,59.67,21816.40,23,2,9.00,9.64,0.00,0.07


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-10-23') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
444038,2021-10-23,Imperial,California,36071,765.00,6025,CA,County,181215,19905.09,422.15,422.15,19905.09,1,3,0.00,60.43,0.00,0.43
3192,2021-10-23,Los Angeles,California,1484896,26540.00,6037,CA,County,10039107,14791.12,264.37,264.37,14791.12,2,12,1161.00,1028.71,13.00,16.57
220408,2021-10-23,San Bernardino,California,358677,5692.00,6071,CA,County,2180085,16452.43,261.09,261.09,16452.43,3,5,0.00,310.50,0.00,9.64
98225,2021-10-23,Stanislaus,California,86299,1335.00,6099,CA,County,550660,15671.92,242.44,242.44,15671.92,4,7,0.00,163.50,0.00,2.43
779857,2021-10-23,Inyo,California,2142,41.00,6027,CA,County,18039,11874.27,227.29,227.29,11874.27,5,24,0.00,21.79,0.00,0.14
78065,2021-10-23,San Joaquin,California,102776,1731.00,6077,CA,County,762148,13485.04,227.12,227.12,13485.04,6,18,0.00,139.00,0.00,3.57
946760,2021-10-23,Kings,California,32938,329.00,6031,CA,County,152940,21536.55,215.12,215.12,21536.55,7,2,0.00,82.21,0.00,1.50
706341,2021-10-23,Merced,California,42811,586.00,6047,CA,County,277680,15417.39,211.03,211.03,15417.39,8,9,0.00,86.57,0.00,1.14
121902,2021-10-23,Tulare,California,80375,978.00,6107,CA,County,466195,17240.64,209.78,209.78,17240.64,9,4,0.00,203.36,0.00,3.29
44150,2021-10-23,Fresno,California,146318,2094.00,6019,CA,County,999101,14644.97,209.59,209.59,14644.97,10,14,0.00,306.64,0.00,5.43


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
1663099,2021-10-23,Lassen,California,7543,49.00,6035,CA,County,30573,24672.10,160.27,160.27,24672.10,22,1,0.00,16.07,0.00,0.71
946760,2021-10-23,Kings,California,32938,329.00,6031,CA,County,152940,21536.55,215.12,215.12,21536.55,7,2,0.00,82.21,0.00,1.50
444038,2021-10-23,Imperial,California,36071,765.00,6025,CA,County,181215,19905.09,422.15,422.15,19905.09,1,3,0.00,60.43,0.00,0.43
121902,2021-10-23,Tulare,California,80375,978.00,6107,CA,County,466195,17240.64,209.78,209.78,17240.64,9,4,0.00,203.36,0.00,3.29
220408,2021-10-23,San Bernardino,California,358677,5692.00,6071,CA,County,2180085,16452.43,261.09,261.09,16452.43,3,5,0.00,310.50,0.00,9.64
277389,2021-10-23,Kern,California,146348,1648.00,6029,CA,County,900202,16257.24,183.07,183.07,16257.24,14,6,0.00,318.43,0.00,4.71
98225,2021-10-23,Stanislaus,California,86299,1335.00,6099,CA,County,550660,15671.92,242.44,242.44,15671.92,4,7,0.00,163.50,0.00,2.43
1011844,2021-10-23,Glenn,California,4384,42.00,6021,CA,County,28393,15440.43,147.92,147.92,15440.43,25,8,0.00,11.14,0.00,0.21
706341,2021-10-23,Merced,California,42811,586.00,6047,CA,County,277680,15417.39,211.03,211.03,15417.39,8,9,0.00,86.57,0.00,1.14
44746,2021-10-23,Madera,California,23955,285.00,6039,CA,County,157327,15226.25,181.15,181.15,15226.25,15,10,0.00,64.07,0.00,0.71


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
444038,2021-10-23,Imperial,California,36071,765.00,6025,CA,County,181215,19905.09,422.15,422.15,19905.09,1,3,0.00,60.43,0.00,0.43
3192,2021-10-23,Los Angeles,California,1484896,26540.00,6037,CA,County,10039107,14791.12,264.37,264.37,14791.12,2,12,1161.00,1028.71,13.00,16.57
220408,2021-10-23,San Bernardino,California,358677,5692.00,6071,CA,County,2180085,16452.43,261.09,261.09,16452.43,3,5,0.00,310.50,0.00,9.64
98225,2021-10-23,Stanislaus,California,86299,1335.00,6099,CA,County,550660,15671.92,242.44,242.44,15671.92,4,7,0.00,163.50,0.00,2.43
779857,2021-10-23,Inyo,California,2142,41.00,6027,CA,County,18039,11874.27,227.29,227.29,11874.27,5,24,0.00,21.79,0.00,0.14
78065,2021-10-23,San Joaquin,California,102776,1731.00,6077,CA,County,762148,13485.04,227.12,227.12,13485.04,6,18,0.00,139.00,0.00,3.57
946760,2021-10-23,Kings,California,32938,329.00,6031,CA,County,152940,21536.55,215.12,215.12,21536.55,7,2,0.00,82.21,0.00,1.50
706341,2021-10-23,Merced,California,42811,586.00,6047,CA,County,277680,15417.39,211.03,211.03,15417.39,8,9,0.00,86.57,0.00,1.14
121902,2021-10-23,Tulare,California,80375,978.00,6107,CA,County,466195,17240.64,209.78,209.78,17240.64,9,4,0.00,203.36,0.00,3.29
44150,2021-10-23,Fresno,California,146318,2094.00,6019,CA,County,999101,14644.97,209.59,209.59,14644.97,10,14,0.00,306.64,0.00,5.43


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'current_crate', 'state_case', 'case_day', '14day_case_avg']]
top10_ca_case_df

,date,county,cases,current_crate,state_case,case_day,14day_case_avg
0,03/20/20,Imperial,4,2.21,3,4.00,82.50
1,03/21/20,Imperial,4,2.21,3,0.00,66.21
2,03/22/20,Imperial,4,2.21,3,0.00,62.07
3,03/23/20,Imperial,4,2.21,3,0.00,60.64
4,03/24/20,Imperial,9,4.97,3,5.00,53.57
...,...,...,...,...,...,...,...
5780,10/19/21,Glenn,4342,15292.50,8,6.00,9.57
5781,10/20/21,Glenn,4352,15327.72,8,10.00,10.29
5782,10/21/21,Glenn,4367,15380.55,8,15.00,10.43
5783,10/22/21,Glenn,4384,15440.43,8,17.00,11.14


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','deaths', 'current_drate', 'state_death', 'death_day', '14day_death_avg']]
top10_ca_death_df

,date,county,deaths,current_drate,state_death,death_day,14day_death_avg
0,03/20/20,Imperial,0.00,0.00,1,0.00,0.07
1,03/21/20,Imperial,0.00,0.00,1,0.00,0.07
2,03/22/20,Imperial,0.00,0.00,1,0.00,0.07
3,03/23/20,Imperial,0.00,0.00,1,0.00,0.07
4,03/24/20,Imperial,0.00,0.00,1,0.00,0.07
...,...,...,...,...,...,...,...
5908,10/19/21,Fresno,2063.00,206.49,10,0.00,6.07
5909,10/20/21,Fresno,2073.00,207.49,10,10.00,6.14
5910,10/21/21,Fresno,2084.00,208.59,10,11.00,5.43
5911,10/22/21,Fresno,2094.00,209.59,10,10.00,5.43


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)